<a href="https://colab.research.google.com/github/takuxp/bigdata_study/blob/main/7%ED%9A%8C_%EA%B8%B0%EC%B6%9C_%EC%9C%A0%ED%98%95(%EC%9E%91%EC%97%85%ED%98%953)_%ED%92%80%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 작업형3

### 문제1. 주어진 조개 데이터 300개 중 앞에서부터 210개는 train 데이터로 만들고, 나머지 90개는 test데이터로 만든다. 모델을 학습(적합)할 때는 train데이터를 사용하고, 예측할 때는 test데이터를 사용한다. 모델은 로지스틱 회귀를 써서 성별(gender)을 예측하되, 패널티는 부과하지 않는다.
- 종속변수: gender(0:암컷, 1:수컷)

### 문제1-1. weight를 독립변수로 gender를 종속변수로 사용하여 로지스틱 회귀 모형을 만들고, weight 변수가 한 단위 증가할 때 수컷일 오즈비 값은? (반올림하여 소수 넷째자리까지 계산)

In [ ]:
import pandas as pd
df = pd.read_csv("clam.csv")

# 데이터셋 분할
train = df.iloc[:210]
test = df.iloc[210:]
print(train.shape, test.shape)

(210, 6) (90, 6)


In [ ]:
from statsmodels.formula.api import logit
import numpy as np

# 1) 로지스틱 회귀 모델 생성 및 학습
model = logit("gender ~ weight", data=train).fit()
print(model.summary())

# 2) 오즈비 계산
print(round(np.exp(0.0047),4))
print(round(np.exp(model.params['weight']),4))

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                 gender   No. Observations:                  210
Model:                          Logit   Df Residuals:                      208
Method:                           MLE   Df Model:                            1
Date:                Wed, 19 Jun 2024   Pseudo R-squ.:                0.003431
Time:                        17:32:13   Log-Likelihood:                -144.91
converged:                       True   LL-Null:                       -145.41
Covariance Type:            nonrobust   LLR p-value:                    0.3178
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3140      0.276     -1.137      0.256      -0.855       0.227
weight         0.0047      0.

In [ ]:
# [참고] <심화학습> 암컷의 오즈비
female_odds_ratio = round(np.exp(-model.params['weight']), 4)
print(female_odds_ratio)

0.9953


### 문제1-2. gender를 종속변수로 하고 나머지 변수들(age, length, diameter, height, weight)을 독립변수로 사용하는 로지스틱 회귀 모델을 적합시킨 후, 잔차 이탈도(Residual Deviance)를 계산하시오. (반올림하여 소수 둘째자리까지 계산)

In [ ]:
# 방법1
from statsmodels.formula.api import logit

# 1) 로지스틱 회귀 모델 생성 및 학습
model = logit("gender ~ age + length + diameter + height + weight", data=train).fit()
print(model.summary())

# 2) 잔차 이탈도 계산
print(round(-2 * model.llf,2))

Optimization terminated successfully.
         Current function value: 0.683173
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                 gender   No. Observations:                  210
Model:                          Logit   Df Residuals:                      204
Method:                           MLE   Df Model:                            5
Date:                Wed, 19 Jun 2024   Pseudo R-squ.:                 0.01336
Time:                        17:33:45   Log-Likelihood:                -143.47
converged:                       True   LL-Null:                       -145.41
Covariance Type:            nonrobust   LLR p-value:                    0.5662
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6701      0.651      1.030      0.303      -0.605       1.946
age           -0.0482      0.

In [ ]:
# 방법2
from statsmodels.formula.api import glm
import statsmodels.api as sm

# 1) glm 모델 적합 (로지스틱 회귀를 위해 이항 분포 사용)
formula = "gender ~ age + length + diameter + height + weight"
model = glm(formula, data=train, family=sm.families.Binomial()).fit()

# 2) 잔차이탈도 계산
print(model.summary())
print(round(model.deviance,2))

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 gender   No. Observations:                  210
Model:                            GLM   Df Residuals:                      204
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -143.47
Date:                Wed, 19 Jun 2024   Deviance:                       286.93
Time:                        17:33:55   Pearson chi2:                     210.
No. Iterations:                     4   Pseudo R-squ. (CS):            0.01833
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6701      0.651      1.030      0.3

### 문제1-3. 독립변수 weight 만 사용해 학습한 로지스틱 회귀모델에서 test데이터의 gender를 예측하고, error rate(오류율)를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
from sklearn.metrics import accuracy_score
from statsmodels.formula.api import logit

# 데이터셋 분할
train = df.iloc[:210]
test = df.iloc[210:]

# 1) 학습, test데이터를 사용해 예측 (0.5 미만: 0, 0.5 이상 1)
model = logit("gender ~ weight", data=train).fit()
target = test.pop("gender")
pred = model.predict(test) > 0.5

# 2) 실제 값과 예측 값을 사용하여 정확도 계산
acc = accuracy_score(target, pred)

# 3) 오류율 계산
print(round(1-acc,3))

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
0.478


In [ ]:
# 다른 풀이 (사이킷런)
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

train = df.iloc[:210]
test = df.iloc[210:]

# 1) 학습 데이터와 테스트 데이터 분리
X_train = train[['weight']]
y_train = train['gender']
X_test = test[['weight']]
y_test = test['gender']

# 2) 로지스틱 회귀 모델 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 3) 테스트 데이터를 사용해 예측
pred = model.predict(X_test)

# 4) 실제 값과 예측값을 사용하여 정확도 계산
acc = accuracy_score(y_test, pred)

# 5) 오류율 계산
print(round(1 - acc, 3))

0.478


### 문제2-1. ERP와 가장 상관관계 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
import pandas as pd
df = pd.read_csv("system.csv")
df.head(3)

,ERP,Feature1,Feature2,Feature3,CPU
0,135.281047,235.088034,44.462272,-36.532634,20.492369
1,108.003144,36.571914,46.409312,-21.709347,34.169811
2,119.574760,52.239055,66.494894,-30.590820,67.624248


In [ ]:
import pandas as pd
df = pd.read_csv("system.csv")

# 1) ERP와 각 변수 사이의 상관계수 계산
df.corr()
# 0.434

,ERP,Feature1,Feature2,Feature3,CPU
ERP,1.000000,0.434442,0.076116,-0.059888,-0.201027
Feature1,0.434442,1.000000,0.076148,0.049980,-0.098571
Feature2,0.076116,0.076148,1.000000,0.126589,0.071456
Feature3,-0.059888,0.049980,0.126589,1.000000,-0.035247
CPU,-0.201027,-0.098571,0.071456,-0.035247,1.000000


### 문제2-2. CPU 컬럼이 100미만인 것만 찾아 ERP를 종속 변수로, 나머지 변수들을 독립 변수로 설정해 선형회귀 모델을 만들고 적합한 결정계수를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
from statsmodels.formula.api import ols

# 1) CPU가 100 미만인 데이터 필터링
cond = df['CPU'] < 100
df = df[cond]

# 2) 선형회귀 모델 생성: ERP를 종속 변수로, 나머지 변수들을 독립 변수로 설정
model = ols('ERP ~ Feature1 + Feature2 + Feature3 + CPU', data=df).fit()

# 3) 모델 요약 정보 출력
print(model.summary())
print(model.rsquared)
print(model.rsquared_adj)

                            OLS Regression Results                            
Dep. Variable:                    ERP   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     6.937
Date:                Wed, 19 Jun 2024   Prob (F-statistic):           6.07e-05
Time:                        17:32:17   Log-Likelihood:                -429.44
No. Observations:                 100   AIC:                             868.9
Df Residuals:                      95   BIC:                             881.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     74.4348     15.094      4.932      0.0

### 문제 2-3. 2-2에서 만든 모델에서 독립변수 중 p-value가 가장 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
# pvalue
# 0.457
print(model.pvalues)

Intercept    0.000003
Feature1     0.000014
Feature2     0.456708
Feature3     0.298206
CPU          0.067968
dtype: float64


In [ ]:
import pandas as pd
df = pd.read_csv("system.csv")
sum(df['CPU'] < 100)
df

,ERP,Feature1,Feature2,Feature3,CPU
0,135.281047,235.088034,44.462272,-36.532634,20.492369
1,108.003144,36.571914,46.409312,-21.709347,34.169811
2,119.574760,52.239055,66.494894,-30.590820,67.624248
3,144.817864,196.195890,59.828956,-33.400891,87.923476
4,137.351160,75.175619,59.601973,-26.668085,54.367805
...,...,...,...,...,...
95,114.131463,105.039508,67.053370,-33.807461,33.747738
96,100.210000,141.114900,51.465875,-25.710380,97.049369
97,135.717410,179.363130,58.744305,-24.294491,13.343943
98,102.538242,217.189747,44.008265,-22.667106,9.680395
